In [112]:
import pandas as pd
import numpy as np

tickers = ['aapl', 'amzn', 'googl', 'msft', 'tsla']
df_dict = {}
for ticker in tickers:
    df = pd.read_csv(f"{ticker}.us.txt")
    df["Date"] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df_dict[ticker.upper()] = df

combined_df = pd.concat(df_dict.values(), keys=df_dict.keys(), names = ['Ticker'])
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 26691 entries, ('AAPL', Timestamp('1984-09-07 00:00:00')) to ('TSLA', Timestamp('2017-11-10 00:00:00'))
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Open     26691 non-null  float64
 1   High     26691 non-null  float64
 2   Low      26691 non-null  float64
 3   Close    26691 non-null  float64
 4   Volume   26691 non-null  int64  
 5   OpenInt  26691 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 1.6+ MB


In [113]:
missing_summary = combined_df.isna().groupby('Ticker').sum()
print(missing_summary)

        Open  High  Low  Close  Volume  OpenInt
Ticker                                         
AAPL       0     0    0      0       0        0
AMZN       0     0    0      0       0        0
GOOGL      0     0    0      0       0        0
MSFT       0     0    0      0       0        0
TSLA       0     0    0      0       0        0


We can see that there are no NA values in our data. Thus we can proceed safely.

In [114]:
# Taking 10 years data only
end_date = combined_df.index.get_level_values('Date').max()
start_date = end_date - pd.DateOffset(years=10)
combined_df = combined_df.loc[(slice(None), slice(start_date, end_date)), :]

In [115]:
# Adding columns for daily return, 7-day moving average, 30-day moving average, and rolling volatility (30d)
def add_features(df):
    df = df.copy()
    df['Daily Return'] = df['Close'].pct_change()
    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['Volatility_30d'] = df['Daily Return'].rolling(window=30).std()
    return df

combined_df = combined_df.groupby(level='Ticker', group_keys=False).apply(add_features)
# remove starting rows which contain NA
combined_df = combined_df.dropna()

In [116]:
avg_return = combined_df.groupby(level='Ticker')['Daily Return'].mean()
avg_return

Ticker
AAPL     0.000969
AMZN     0.001305
GOOGL    0.000609
MSFT     0.000558
TSLA     0.001994
Name: Daily Return, dtype: float64

We can see that Tesla has the highest average return of approximately 0.2%

In [117]:
max_volatility_30d = combined_df.groupby(level='Ticker')['Volatility_30d'].idxmax().apply(
    lambda idx: pd.Series({
        'Max 30-day Rolling Volatility': combined_df.loc[idx, 'Volatility_30d'],
        'End date of 30-day period': idx[1]
    })
).reset_index()
max_volatility_30d

,Ticker,Max 30-day Rolling Volatility,End date of 30-day period
0,AAPL,0.066955,2008-10-28
1,AMZN,0.069423,2008-10-28
2,GOOGL,0.055522,2008-10-28
3,MSFT,0.060814,2008-11-21
4,TSLA,0.094450,2010-08-10


We can see that again, Tesla has the highest 30-day rolling volatility, on 10th Aug 2010.